## Check the API key


In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
print(os.getenv("GROQ_API_KEY"))

## do simple LLM API call

In [5]:
from langchain_groq import ChatGroq
llm = ChatGroq(model = "openai/gpt-oss-120b", api_key = os.getenv("GROQ_API_KEY"))
print(llm.predict("Tell me a joke about computers."))


Why did the computer go to therapy?

Because it had too many *bytes* of emotional baggage and kept crashing under the pressure! 😄


## Prompting

In [6]:
from langchain_core.messages import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

## System message
* It defines who the model is, not what it must say
* SystemMessage has highest priority in the prompt stack
* It sets behavioral constraints 


In [9]:
SystemMessage(content="You are a strict physics professor.")

SystemMessage(content='You are a strict physics professor.', additional_kwargs={}, response_metadata={})

## Human Message
* This represents the user request 
* This model is weighted heavily but below system message

In [7]:
HumanMessage(content="Explain Navier-Stokes simply.")

HumanMessage(content='Explain Navier-Stokes simply.', additional_kwargs={}, response_metadata={})

## AIResponse - Model output
* Without AIMessage objects, multi-turn reasoning breaks.
* memory can be injected
* Context can be extended


In [8]:
AIMessage(content="Navier-Stokes equations describe fluid motion...")

AIMessage(content='Navier-Stokes equations describe fluid motion...', additional_kwargs={}, response_metadata={})

## Short demo working

In [11]:
from langchain_groq import ChatGroq         # Import ChatGroq LLM wrapper
from langchain_core.messages import SystemMessage, HumanMessage     # Import message types

llm = ChatGroq(model="openai/gpt-oss-120b", temperature=0)   # Initialize LLM with specific model and temperature

messages = [    
    SystemMessage(content="You are a very strict math tutor."),     # System message to set the context
    HumanMessage(content="Explain gradient descent in one paragraph.")      # Human message with the user's query
]

response = llm.invoke(messages)  # Invoke the LLM with the messages
print(response.content)  # Print the LLM's response content 

Gradient descent is an iterative optimization algorithm used to find a local minimum of a differentiable function \(f(\mathbf{x})\); starting from an initial guess \(\mathbf{x}_0\), it updates the parameters by moving them in the direction of the steepest decrease, namely the negative gradient, according to the rule \(\mathbf{x}_{k+1} = \mathbf{x}_k - \alpha_k \nabla f(\mathbf{x}_k)\), where \(\alpha_k>0\) is a step‑size (or learning rate) that may be constant or adaptively chosen; each iteration reduces the function value (provided \(\alpha_k\) is suitably small), and under appropriate conditions—convexity of \(f\) and a diminishing step‑size satisfying \(\sum_k \alpha_k = \infty\) and \(\sum_k \alpha_k^2 < \infty\)—the sequence \(\{\mathbf{x}_k\}\) converges to a global minimizer, while for non‑convex functions it converges to a critical point (which may be a local minimum, saddle point, or maximum).


What happened internally 

* Langchain validated each message object
* Converted it into groq's json schema 
* Groq executed the model
* Returned json object 
```
{"role": "assistant", "content": " ..."}
```
* Langchain wrapped it as AIMessage

## Chat Prompt Template - Formating the message

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a strict {subject} tutor."),    # System message with a placeholder
    ("human", "Explain {topic} in {difficulty} terms.")     # Human message with placeholders
])
prompt 


ChatPromptTemplate(input_variables=['difficulty', 'subject', 'topic'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['subject'], input_types={}, partial_variables={}, template='You are a strict {subject} tutor.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['difficulty', 'topic'], input_types={}, partial_variables={}, template='Explain {topic} in {difficulty} terms.'), additional_kwargs={})])

What this object actually is 
* Stores the message blueprints
* Tracks required variables
* Validates inputs before execution
* Produced message objects not strings

In [30]:
message = prompt.format_messages(
    subject="physics", 
    topic="quantum entanglement", 
    difficulty="simple")      # Format messages with specific values

for m in message:
    print(type(m) ,"---> ", m.content)  
print()    # Map formatted messages to their types and content
print("This is SystemMessage",message[0],"\n")
print("This is HumanMessage",message[1],"\n")
print(message)

<class 'langchain_core.messages.system.SystemMessage'> --->  You are a strict physics tutor.
<class 'langchain_core.messages.human.HumanMessage'> --->  Explain quantum entanglement in simple terms.

This is SystemMessage content='You are a strict physics tutor.' additional_kwargs={} response_metadata={} 

This is HumanMessage content='Explain quantum entanglement in simple terms.' additional_kwargs={} response_metadata={} 

[SystemMessage(content='You are a strict physics tutor.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Explain quantum entanglement in simple terms.', additional_kwargs={}, response_metadata={})]


## Generating output by providing formatted message

In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="openai/gpt-oss-120b", temperature=0)

response = llm.invoke(messages)     
print(response.content)


content='Gradient descent is an iterative optimization algorithm used to find a local minimum of a differentiable function\u202f\\(f(\\mathbf{x})\\); starting from an initial guess \\(\\mathbf{x}_0\\), it updates the parameters by moving them in the direction of the steepest decrease, namely the negative gradient, according to the rule \\(\\mathbf{x}_{k+1} = \\mathbf{x}_k - \\alpha_k \\nabla f(\\mathbf{x}_k)\\), where \\(\\alpha_k>0\\) is a step‑size (or learning rate) that may be constant or adaptively chosen; each iteration reduces the function value (provided \\(\\alpha_k\\) is suitably small), and under appropriate conditions—convexity of \\(f\\) and a diminishing step‑size satisfying \\(\\sum_k \\alpha_k = \\infty\\) and \\(\\sum_k \\alpha_k^2 < \\infty\\)—the sequence \\(\\{\\mathbf{x}_k\\}\\) converges to a global minimizer, while for non‑convex functions it converges to a critical point (which may be a local minimum, saddle point, or maximum).' additional_kwargs={} response_met

* Here the the messages object is passed to the llm invoke
* Langchain chatPromptTemplate converts those objects into groq accepted json schema
* Then that is passed to the groq model for generation
* The model outputs the contents and meta data
* Response recieves the llm response object and prints the content of the llm response

## Final takeaways 
<b> ChatPromptTemplate </b> is a schema for the llm to understand the context, user message etc easily


## LCEL


In [ ]:
# import runnablemap , runnablelambda
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate   
from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.runnables import RunnableLambda,RunnableMap
import os
from dotenv import load_dotenv
load_dotenv()  # Load environment variables from .env file
llm = ChatGroq(model="openai/gpt-oss-120b", temperature=0, api_key=os.getenv("GROQ_API_KEY"))   # Initialize LLM with specific model and temperature

human_message = HumanMessagePromptTemplate.from_template("Explain {topic} in {difficulty} terms.",input_variables=["topic","difficulty"])
system_message = SystemMessagePromptTemplate.from_template("you are a strict {subject} tutor.",input_variables=["subject"])


prompt1 = ChatPromptTemplate.from_messages([
  human_message, system_message
  ])

chain = (
    prompt1 | 
    llm      # Convert response to uppercase
)
final_response = chain.invoke({"topic": "calculus", "difficulty": "beginner", "subject": "math"})
print("Final Response:")
print(final_response) 



Final Response:
content="**Calculus\u202f—\u202fA Beginner’s Guide**\n\n---\n\n### 1. What Is Calculus, Anyway?\nCalculus is the branch of mathematics that studies **change** and **accumulation**.  \n- **Change** → How something varies moment‑by‑moment (think speed, slope, growth).  \n- **Accumulation** → How small pieces add up to a whole (think distance traveled, area under a curve).\n\nThese two ideas are captured by the **derivative** and the **integral**, which turn out to be two sides of the same coin (the Fundamental Theorem of Calculus).\n\n---\n\n## 2. The Building Block: Limits\nBefore we can talk about derivatives or integrals we need the notion of a **limit**.\n\n**Intuition:**  \nImagine you have a function \\(f(x)\\) and you want to know what value it “approaches” as \\(x\\) gets very close to a certain number \\(a\\). The limit tells you that.\n\n- Notation: \\(\\displaystyle \\lim_{x\\to a} f(x) = L\\) means “as \\(x\\) gets arbitrarily close to \\(a\\), \\(f(x)\\) gets

In [9]:
print(final_response.content)

**Calculus — A Beginner’s Guide**

---

### 1. What Is Calculus, Anyway?
Calculus is the branch of mathematics that studies **change** and **accumulation**.  
- **Change** → How something varies moment‑by‑moment (think speed, slope, growth).  
- **Accumulation** → How small pieces add up to a whole (think distance traveled, area under a curve).

These two ideas are captured by the **derivative** and the **integral**, which turn out to be two sides of the same coin (the Fundamental Theorem of Calculus).

---

## 2. The Building Block: Limits
Before we can talk about derivatives or integrals we need the notion of a **limit**.

**Intuition:**  
Imagine you have a function \(f(x)\) and you want to know what value it “approaches” as \(x\) gets very close to a certain number \(a\). The limit tells you that.

- Notation: \(\displaystyle \lim_{x\to a} f(x) = L\) means “as \(x\) gets arbitrarily close to \(a\), \(f(x)\) gets arbitrarily close to \(L\).”

**Why it matters:**  
Limits let us hand

Use of RunnableMap and RunnableLambda

In [14]:
from langchain_core.runnables import RunnableLambda,RunnableMap

mapped_input = RunnableMap({
    "topic": lambda x: x["topic"],
    "difficulty": lambda x: x["difficulty"],
    "subject": lambda x: x["subject"]
})
mapped_output = RunnableLambda(
    lambda x: x.content
)


chain2 = (
    mapped_input |
    prompt1 |
    llm |
    mapped_output
)

final_response = chain2.invoke({"topic": "calculus", "difficulty": "beginner", "subject": "math"})
print("Final Response with RunnableMap and RunnableLambda:")
print(final_response)

Final Response with RunnableMap and RunnableLambda:
**Calculus – The Essentials (and What You Must Master)**  

---

### 1. Why Calculus Exists  
- **Goal:** Describe how things change.  
- **Two big questions:**  
  1. *Instantaneous* change – “How fast is this moving right now?” → **Derivative**.  
  2. *Accumulated* change – “How much total distance has been covered?” → **Integral**.  

If you can’t answer these, you haven’t learned calculus.

---

### 2. The Building Block: Limits  
- **Definition (no hand‑waving):**  
  \[
  \lim_{x\to a} f(x)=L
  \]
  means: as the input \(x\) gets arbitrarily close to \(a\) (but not equal), the output \(f(x)\) gets arbitrarily close to \(L\).  

- **Why it matters:** Limits let us talk about “what happens at a point” even when the function misbehaves there (e.g., division by zero).  

- **You must be able to compute simple limits** using:
  - Direct substitution (if the function is continuous).  
  - Factoring and canceling.  
  - Rationalizing 

## Use the StrOutputParser rather then RunnableLambda

In [16]:
from langchain_core.runnables import RunnableMap
from langchain_core.output_parsers import StrOutputParser

mapped_input = RunnableMap({
    "topic": lambda x: x["topic"],
    "difficulty": lambda x: x["difficulty"],
    "subject": lambda x: x["subject"]
})
mapped_output = StrOutputParser()


chain2 = (
    mapped_input |
    prompt1 |
    llm |
    mapped_output
)

final_response = chain2.invoke({"topic": "calculus", "difficulty": "beginner", "subject": "math"})
print("Final Response with RunnableMap and StrOutputParser:")
print(final_response)

Final Response with RunnableMap and StrOutputParser:
**Calculus — A Beginner’s Guide**

---

### 1. What Is Calculus, Anyway?
Calculus is the branch of mathematics that studies **change** and **accumulation**.  
- **Change** → How something varies moment‑by‑moment (think speed, slope, growth).  
- **Accumulation** → How small pieces add up to a whole (think distance traveled, area under a curve).

These two ideas are captured by the **derivative** and the **integral**, which turn out to be two sides of the same coin (the Fundamental Theorem of Calculus).

---

## 2. The Building Block: Limits
Before we can talk about derivatives or integrals we need the notion of a **limit**.

**Intuition:**  
Imagine you have a function \(f(x)\) and you want to know what value it “approaches” as \(x\) gets very close to a certain number \(a\). The limit tells you that.

- Notation: \(\displaystyle \lim_{x\to a} f(x) = L\) means “as \(x\) gets arbitrarily close to \(a\), \(f(x)\) gets arbitrarily close